## FastAPI

In [ ]:
# $ pip install fastapi[all]

In [ ]:
# app.py (basic app)

from fastapi import FastAPI 

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}

# run with: $ uvicorn app:app --reload
# go to http://localhost:8000/

In [ ]:
# app.py (app to predict digit on an image)

from fastapi import FastAPI, File, UploadFile
from PIL import Image
import onnxruntime as ort 
import numpy as np
import io
import math

app = FastAPI()

ort_session = ort.InferenceSession('models/binary_classifier_3.onnx')
TRHESHOLD = 0.5

@app.get("/")
async def root():
    return {"message": "Hello World"}

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    request_object_content = await file.read()
    img = Image.open(io.BytesIO(request_object_content))
    input = np.expand_dims(np.array(img, dtype=np.uint8), axis=0)
    ort_inputs = {
        "input": input
    }
    ort_output = ort_session.run(['output'], ort_inputs)[0]
    output = sigmoid(ort_output)
    return {
        "proba": output,
        "label": "3" if output > TRHESHOLD else "no 3"
    }

# Check the automatic generated docs of the API, where you can see every endopoint, parameters
# description and even the possibility of testing it:
# http://localhost:8000/docs

## Docker

In [ ]:
# Install docker

# Create a Dockerfile with:

"""
FROM continuumio/miniconda3

RUN  conda install y -c conda-forge \
pillow \
    onnxruntime \
    fastapi \
    uvicorn \
    python-multipart 

COPY ./models /models
COPY ./app.py /app.py

CMD uvicorn app:app --host=0.0.0.0 --port=$PORT
"""

# Test it on your local env:
# $ docker build -t dlops .
# $ docker run -p 8000:8000 -e PORT=8000 dlops

## Heroku

In [ ]:
# Install the Heroku CLI, then:

# $ heroku login
# $ heroku container:login
# $ heroku container:push web -a <heroku-app-name>
# $ heroku container:release web -a <heroku-app-name>
